<a href="https://colab.research.google.com/github/Muhire-taa/Forex-makert-trading-advisory-and-forecasting/blob/main/PPO_ForexGYMEnv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29413 sha256=460e718053e6dc67c3520b9e492a1ae07604dc1e3e7454fe71f822c91149c5bc
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import random
import json
import math
import gym
from gym import spaces
import pandas as pd
import numpy as np
import ta
# import talib
import tensorflow as tf
from tqdm import tqdm
from collections import deque

In [ ]:
file_path = "/content/drive/MyDrive/output_4H.csv"

In [ ]:
def dataset_loader(file_path):
    df = pd.read_csv(file_path, date_parser=True)
    df = df.sort_values('time', ascending=True)
    df = df.set_index('time')
    df["rsi"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range()
    bb = ta.volatility.BollingerBands(df['close'], window=20)
    df['upper_bb'] = bb.bollinger_hband()
    df['lower_bb'] = bb.bollinger_lband()
    # Calculate the Stochastic Oscillator using ta library
    stoch = ta.momentum.StochasticOscillator(high=df['high'], low=df['low'], close=df['close'], window=14)
    # Add %K and %D to dataframe
    df['%K'] = stoch.stoch()
    df['%D'] = stoch.stoch_signal()
    ema_slow = df['close'].ewm(span=26).mean()
    ema_fast = df['close'].ewm(span=12).mean()
    # Calculate the MACD line
    macd_line = ema_fast - ema_slow

    # Calculate the signal line (EMA of MACD line)
    signal_line = macd_line.ewm(span=9).mean()

    # Calculate the histogram
    histogram = macd_line - signal_line

    # Add the MACD indicators to the DataFrame
    df['macd_line'] = macd_line
    df['histogram'] = histogram
    ema_21 = ta.trend.EMAIndicator(close=df['close'], window=21, fillna=False)
    df['EMA_21'] = ema_21.ema_indicator()
    df['change'] = df['close'] - df['open']
    # Calculate Daily Return (Close-to-Close Change)
    df['Daily_Return'] = df['close'].pct_change() * 100

    # Calculate High-Low Range
    df['High_Low_Range'] = df['high'] - df['low']

    df.dropna(inplace=True)
    df=df.drop(columns=['tick_volume','spread','real_volume'])

    return df


In [ ]:
df=dataset_loader(file_path)
df

,open,high,low,close,rsi,atr,upper_bb,lower_bb,%K,%D,macd_line,histogram,EMA_21,change,Daily_Return,High_Low_Range
time,,,,,,,,,,,,,,,,
1977-08-12 00:00:00,0.83870,0.83960,0.83800,0.83880,28.290666,0.003555,0.875135,0.837065,2.439024,1.172720,-0.003790,-0.001276,0.852191,0.00010,-0.533618,0.00160
1977-08-15 00:00:00,0.83760,0.83760,0.83760,0.83760,27.312840,0.003386,0.875158,0.834472,0.000000,1.172720,-0.004183,-0.001333,0.850865,0.00000,-0.143062,0.00000
1977-08-16 00:00:00,0.83630,0.83650,0.83590,0.83640,26.332677,0.003266,0.875028,0.831992,2.074689,1.504571,-0.004519,-0.001334,0.849550,0.00010,-0.143266,0.00060
1977-08-17 00:00:00,0.84010,0.84050,0.83970,0.84010,34.176332,0.003326,0.873560,0.830800,22.826087,8.300259,-0.004473,-0.001029,0.848691,0.00000,0.442372,0.00080
1977-08-18 00:00:00,0.84030,0.84030,0.84030,0.84030,34.581799,0.003102,0.872301,0.829719,24.858757,16.586511,-0.004371,-0.000740,0.847928,0.00000,0.023807,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-23 20:00:00,1.05854,1.06033,1.05810,1.05971,37.859271,0.002857,1.070081,1.057881,16.501650,14.224359,-0.002267,-0.000404,1.063331,0.00117,0.110530,0.00223
2023-02-24 00:00:00,1.05926,1.06143,1.05900,1.06013,39.252363,0.002826,1.069499,1.057620,27.848101,17.065979,-0.002229,-0.000293,1.063040,0.00087,0.039633,0.00243
2023-02-24 04:00:00,1.06012,1.06063,1.05947,1.05975,38.413282,0.002707,1.068818,1.057423,23.475259,22.608337,-0.002204,-0.000215,1.062741,-0.00037,-0.035845,0.00116


In [ ]:
class ForexTradingEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, df, window_size):
        super(ForexTradingEnv, self).__init__()

        # self.df = df.reset_index()
        if 'time' in df.columns:
            df = df.set_index('time')
        self.df = df
        self.window_size = window_size
        self.current_step = window_size
        self.total_profit = 0
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=0, high=1, shape=(window_size, len(df.columns)), dtype=np.float32)

    def _next_observation(self):
        frame = np.array([self.df.iloc[self.current_step - self.window_size:self.current_step].values])
        obs = frame.reshape((1, self.window_size, self.df.shape[1]))
        return obs



    def _take_action(self, action):
        # Calculate the candlestick body size
        body_size = abs(self.df['close'].iloc[self.current_step] - self.df['open'].iloc[self.current_step]) * 10000
        scaling_factor = max(1, body_size / 10)
        # Initialize reward
        reward = 0
        pip_profit = 0
        if action == 0:  # Hold
            if not self.inventory:
                reward = -body_size
            #print("AI Trader held Position at: ",price_format(self.df['open'].iloc[self.current_step]))

        elif action == 1:
            self.inventory.append(self.df['open'].iloc[self.current_step])

            #print("AI Trader bought at: ", price_format(self.df['open'].iloc[self.current_step]))
        elif action == 2:
            if self.inventory:
                bought_price = self.inventory.pop(0)
                pip_profit = (self.df['close'].iloc[self.current_step]-bought_price) * 10000

                if pip_profit > 0:
                    reward = pip_profit
                else:
                    reward = 2* pip_profit
                #print("AI Trader sold at: ", price_format(self.df['close'].iloc[self.current_step]), " Profit: " + price_format(pip_profit))

        return reward


    def step(self, action):
        reward = self._take_action(action)
        self.current_step += 1
        done = False
        info = {}

        if self.current_step > len(self.df) - 1:
            done = True
            # Calculate total profit for the inventory and add to reward
            total_profit = sum((self.df['close'].iloc[i] - price) * 10000 for i, price in enumerate(self.inventory))
            reward += total_profit
            self.total_profit += total_profit
            self.inventory = []
            info['total_profit'] = self.total_profit
            self.total_profit = 0

        obs = self._next_observation()
        return obs, reward, done, info



    def reset(self):
        self.current_step = self.window_size
        self.inventory = []
        observation = self._next_observation()
        #print("Reset observation shape:", observation.shape)
        return observation

    def render(self, mode='human', close=False):
        profit = sum((self.df['close'].iloc[i] - self.df['open'].iloc[i]) * 10000 for i in range(len(self.inventory)))
        print(f"Step: {self.current_step} - Inventory: {len(self.inventory)} - Profit: {profit}")



In [ ]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))
def price_format(n):
    return "{0:.5f}".format(n)


In [ ]:
class PPOMemory:
    def __init__(self, batch_size):
        self.states = []
        self.probs = []
        self.vals = []
        self.actions = []
        self.rewards = []
        self.dones = []

        self.batch_size = batch_size

    def generate_batches(self):
        n_states = len(self.states)
        batch_start = np.arange(0, n_states, self.batch_size)
        indices = np.arange(n_states, dtype=np.int64)
        np.random.shuffle(indices)
        batches = [indices[i:i+self.batch_size] for i in batch_start]

        return np.array(self.states),\
            np.array(self.actions),\
            np.array(self.probs),\
            np.array(self.vals),\
            np.array(self.rewards),\
            np.array(self.dones),\
            batches

    def store_memory(self, state, action, probs, vals, reward, done):
        reshaped_state = np.squeeze(state, axis=0)
        self.states.append(reshaped_state)
        # self.states.append(state)
        self.actions.append(action)
        self.probs.append(probs)
        self.vals.append(vals)
        self.rewards.append(reward)
        self.dones.append(done)

    def clear_memory(self):
        self.states = []
        self.probs = []
        self.actions = []
        self.rewards = []
        self.dones = []
        self.vals = []

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Dropout

import tensorflow.keras as keras

class ActorNetwork(keras.Model):
    def __init__(self, n_actions, lstm_units_1=256, lstm_units_2=128, fc_dims=128, dropout_rate=0.2):
        super(ActorNetwork, self).__init__()

        self.lstm1 = LSTM(lstm_units_1, return_sequences=True)
        self.dropout1 = Dropout(dropout_rate)
        self.lstm2 = LSTM(lstm_units_2, return_sequences=False)
        self.dropout2 = Dropout(dropout_rate)
        self.fc = Dense(fc_dims, activation='relu')

        self.out = Dense(n_actions, activation='softmax')

    def call(self, state):
        x = self.lstm1(state)
        x = self.dropout1(x)
        x = self.lstm2(x)
        x = self.dropout2(x)
        x = self.fc(x)
        x = self.out(x)

        return x


In [ ]:
class CriticNetwork(keras.Model):
    def __init__(self, lstm_units_1=256, lstm_units_2=128, fc_dims=128, dropout_rate=0.2):
        super(CriticNetwork, self).__init__()
        self.lstm1 = LSTM(lstm_units_1, return_sequences=True)
        self.dropout1 = Dropout(dropout_rate)
        self.lstm2 = LSTM(lstm_units_2, return_sequences=False)
        self.dropout2 = Dropout(dropout_rate)
        self.fc = Dense(fc_dims, activation='relu')
        self.q = Dense(1, activation=None)

    def call(self, state):
        x = self.lstm1(state)
        x = self.dropout1(x)
        x = self.lstm2(x)
        x = self.dropout2(x)
        x = self.fc(x)
        q = self.q(x)

        return q


In [ ]:
# import tensorflow.keras as keras
# from tensorflow.keras.layers import Dense,Flatten


# class ActorNetwork(keras.Model):
#     def __init__(self, n_actions, fc1_dims=256, fc2_dims=512):
#         super(ActorNetwork, self).__init__()

#         self.fc1 = Dense(fc1_dims, activation='relu')
#         self.fc2 = Dense(fc2_dims, activation='relu')
#         self.flatten = Flatten()
#         self.fc3 = Dense(n_actions, activation='softmax')

#     def call(self, state):
#         x = self.fc1(state)
#         x = self.flatten(x)
#         x = self.fc2(x)
#         x = self.fc3(x)

#         return x


# class CriticNetwork(keras.Model):
#     def __init__(self, fc1_dims=256, fc2_dims=512):
#         super(CriticNetwork, self).__init__()
#         self.fc1 = Dense(fc1_dims, activation='relu')
#         self.fc2 = Dense(fc2_dims, activation='relu')
#         self.flatten = Flatten()
#         self.q = Dense(1, activation=None)

#     def call(self, state):
#         x = self.fc1(state)
#         x = self.flatten(x)
#         x = self.fc2(x)
#         q = self.q(x)

#         return q


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp

class Agent:
    def __init__(self, n_actions, input_dims, gamma=0.99, alpha=0.0003,
                 gae_lambda=0.95, policy_clip=0.2, batch_size=64,
                 n_epochs=10, chkpt_dir='models/'):
        self.gamma = gamma
        self.policy_clip = policy_clip
        self.n_epochs = n_epochs
        self.gae_lambda = gae_lambda
        self.chkpt_dir = chkpt_dir

        self.actor = ActorNetwork(n_actions)
        self.actor.compile(optimizer=Adam(learning_rate=alpha))
        self.critic = CriticNetwork()
        self.critic.compile(optimizer=Adam(learning_rate=alpha))

        self.memory = PPOMemory(batch_size)

    def store_transition(self, state, action, probs, vals, reward, done):
        self.memory.store_memory(state, action, probs, vals, reward, done)

    def save_models(self):
        print('... saving models ...')
        self.actor.save(self.chkpt_dir + 'actor')
        self.critic.save(self.chkpt_dir + 'critic')

    def load_models(self):
        print('... loading models ...')
        self.actor = keras.models.load_model(self.chkpt_dir + 'actor')
        self.critic = keras.models.load_model(self.chkpt_dir + 'critic')

    def choose_action(self, observation):
       #print("In choose_action, observation shape:", observation.shape)
        state = observation

        probs = self.actor(state)
        dist = tfp.distributions.Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        value = self.critic(state)

        action = action.numpy()[0]
        value = value.numpy()[0]
        log_prob = log_prob.numpy()[0]

        return action, log_prob, value

    def learn(self):
        for _ in range(self.n_epochs):
            state_arr, action_arr, old_prob_arr, vals_arr,\
                reward_arr, dones_arr, batches = \
                self.memory.generate_batches()

            values = vals_arr
            advantage = np.zeros(len(reward_arr), dtype=np.float32)

            for t in range(len(reward_arr)-1):
                discount = 1
                a_t = 0
                for k in range(t, len(reward_arr)-1):
                    a_t += discount*(reward_arr[k] + self.gamma*values[k+1] * (
                        1-int(dones_arr[k])) - values[k])
                    discount *= self.gamma*self.gae_lambda
                advantage[t] = a_t

            for batch in batches:
                states = tf.convert_to_tensor(state_arr[batch])
                #print("Batch states shape:", states.shape)
                with tf.GradientTape(persistent=True) as tape:
                    states = tf.convert_to_tensor(state_arr[batch])

                    old_probs = tf.convert_to_tensor(old_prob_arr[batch])
                    actions = tf.convert_to_tensor(action_arr[batch])

                    probs = self.actor(states)
                    dist = tfp.distributions.Categorical(probs)
                    new_probs = dist.log_prob(actions)

                    critic_value = self.critic(states)

                    critic_value = tf.squeeze(critic_value, 1)

                    prob_ratio = tf.math.exp(new_probs - old_probs)
                    weighted_probs = advantage[batch] * prob_ratio
                    clipped_probs = tf.clip_by_value(prob_ratio,
                                                     1-self.policy_clip,
                                                     1+self.policy_clip)
                    weighted_clipped_probs = clipped_probs * advantage[batch]
                    actor_loss = -tf.math.minimum(weighted_probs,
                                                  weighted_clipped_probs)
                    actor_loss = tf.math.reduce_mean(actor_loss)

                    returns = advantage[batch] + values[batch]
                    # critic_loss = tf.math.reduce_mean(tf.math.pow(
                    #                                  returns-critic_value, 2))
                    critic_loss = keras.losses.MSE(critic_value, returns)

                actor_params = self.actor.trainable_variables
                actor_grads = tape.gradient(actor_loss, actor_params)
                critic_params = self.critic.trainable_variables
                critic_grads = tape.gradient(critic_loss, critic_params)
                self.actor.optimizer.apply_gradients(
                        zip(actor_grads, actor_params))
                self.critic.optimizer.apply_gradients(
                        zip(critic_grads, critic_params))

        self.memory.clear_memory()

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [ ]:
import matplotlib.pyplot as plt

def plot_learning_curve(x, scores, figure_file):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')
    plt.savefig(figure_file)


In [ ]:
import gym
import tensorflow as tf
from datetime import datetime

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

if __name__ == '__main__':
    df = dataset_loader(file_path)
    env = ForexTradingEnv(df, window_size=90)

    N = 20
    batch_size = 32
    n_epochs = 10
    alpha = 0.0003
    agent = Agent(n_actions=3, batch_size=batch_size,
                  alpha=alpha, n_epochs=n_epochs,
                  input_dims=env.observation_space.shape)
    n_games = 20

    best_score = env.reward_range[0]
    score_history = []
    total_profits=[]
    total_rewards=[]
    learn_iters = 0
    avg_score = 0
    n_steps = 0

    for i in range(n_games):
        observation = env.reset()
        print("Game number: ", i)
        #print("Before choose_action, observation shape:", observation.shape)

        done = False
        score = 0
        episode_profit = 0
        while not done:
            action, prob, val = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            n_steps += 1
            score += reward
            agent.store_transition(observation, action,
                                   prob, val, reward, done)
            if n_steps % N == 0:
                agent.learn()
                learn_iters += 1
                #print(f"Learning triggered at step {n_steps}, total learning iterations: {learn_iters}")
            observation = observation_
            episode_profit = info.get('total_profit', 0)  # Get total_profit from info dictionary

        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        total_profits.append(episode_profit)
        total_rewards.append(score)

        if avg_score > best_score:
            best_score = avg_score
            agent.save_models()
        tf.summary.scalar('reward summary', data=avg_score, step=i)
        # print('episode', i, 'score %.1f' % score, 'avg score %.1f' % avg_score,
        #       'time_steps', n_steps, 'learning_steps', learn_iters)
        env.render()

        # Print total profits and rewards after each episode
        print(f"Episode {i + 1} - Total Profit: {episode_profit}, Total Reward: {score}")

    filename = 'PPO_trading_view.png'
    figure_file = "/content/" + filename
    x = [i + 1 for i in range(len(score_history))]
    plot_learning_curve(x, score_history, figure_file)


Game number:  0


... saving models ...
Step: 43118 - Inventory: 0 - Profit: 0
Episode 1 - Total Profit: -4426.300000000001, Total Reward: -334734.29999999894
Game number:  1
